# BDPBot

---

A bot based on BERT using transfer learning
The dataset to train and test the model will be available of interact in english, but no in spanish

In [1]:
# load bert tokenizer and Classification from transformers
from transformers import BertTokenizer, BertForSequenceClassification

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

e:\BertBot\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
tokenizer_config.json: 100%|██████████| 28.0/28.0 [00:00<00:00, 14.0kB/s]
e:\BertBot\.venv\lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\carlo\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see acti

Now we need to tokenize the texts that are going to be used for train and test the model in this case, i am going to use TriviaQA from http://nlp.cs.washington.edu/triviaqa/

In [2]:
# Download dataset if it doesn't exist

import requests
import tarfile
import os
url = "http://nlp.cs.washington.edu/triviaqa/data/triviaqa-unfiltered.tar.gz"
target_path = 'triviaqa-unfiltered.tar.gz'

if target_path not in os.listdir():
    print("Downloading file")
    # Dowload the file showing the progress bar
    
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        print("Saving file")
        with open(target_path, 'wb') as f:
            f.write(response.raw.read())

    # Unzip the file
    with tarfile.open(target_path, "r:gz") as tar:
        tar.extractall(path=".")
else:
    print("File already exists")

File already exists


In [3]:
# create a dataframe from the json file with the variables that i need 

import pandas as pd
import json

if 'triviaqa-unfiltered.csv' not in os.listdir():
    # Load Json file
    with open('./triviaqa-unfiltered/unfiltered-web-train.json', 'r') as file:
        data = json.load(file)
    if isinstance(data, dict):
        data_to_df = []
        data_json = data["Data"]
        for item in data_json:
            question = item.get('Question', '')  # Usar get para evitar errores si la clave no existe
            answer = item.get('Answer', {}).get('Value', '')  # Anidando get para acceder a 'Value'
            search_results = item.get('SearchResults', [])
            context = ""
            for searched in search_results:
                context += searched.get('Description', '') + " "
            data_to_df.append({
                'Question': question,
                'Answer': answer,
                'Context': context
            })

        # Create a DataFrame from the list of dictionaries
        df = pd.DataFrame(data_to_df)
        # Save the DataFrame to a csv file
        df.to_csv('triviaqa-unfiltered.csv', index=False)
        # Visualizar las primeras filas del DataFrame
        print(df.head())
    else:
        print("Json has not the dxpected format")
else:
    df = pd.read_csv('triviaqa-unfiltered.csv')
    print(df.head())

                                             context  \
0  Welcome to GoComics.com, the world's largest c...   
1  The Nobel Prize in Literature 1930 Sinclair .....   
2  Dame Judi Dench is a renowned ... Born in Engl...   
3  Our expert has answerd your question. Meet our...   
4  Third Man Records Launches First Record Played...   

                                            question  answers  
0  Who was President when the first Peanuts carto...      NaN  
1  Which American-born Sinclair won the Nobel Pri...      NaN  
2         Where in England was Dame Judi Dench born?      NaN  
3  William Christensen of Madison, New Jersey, ha...      NaN  
4  In which decade did Billboard magazine first p...      NaN  


In [4]:
if 'test-triviaqa-unfiltered.csv' not in os.listdir():
    # Load Json file
    with open('./triviaqa-unfiltered/unfiltered-web-dev.json', 'r') as file:
        data = json.load(file)
    if isinstance(data, dict):
        data_to_df = []
        data_json = data["Data"]
        for item in data_json:
            question = item.get('Question', '')  # Usar get para evitar errores si la clave no existe
            answer = item.get('Answer', {}).get('Value', '')  # Anidando get para acceder a 'Value'
            search_results = item.get('SearchResults', [])
            context = ""
            for searched in search_results:
                context += searched.get('Description', '') + " "
            data_to_df.append({
                'Question': question,
                'Answer': answer,
                'Context': context
            })

        # Create a DataFrame from the list of dictionaries
        test = pd.DataFrame(data_to_df)
        # Save the DataFrame to a csv file
        test.to_csv('test-triviaqa-unfiltered.csv', index=False)
        # Visualizar las primeras filas del DataFrame
        print(df.head())
    else:
        print("Json has not the dxpected format")
else:
    test = pd.read_csv('test-triviaqa-unfiltered.csv')
    print(test.head())

                                             context  \
0  Alice Cooper's The Man Behind the Mask Music V...   
1  Jamie Lee Curtis, Actress: True Lies. Jamie Le...   
2  The official website for Andrew Lloyd Webber, ...   
3  The history and complete text of the 1917 Balf...   
4  ... credits and award information for 70 Numbe...   

                                            question             answers  
0              Who was the man behind The Chipmunks?       David Seville  
1                What star sign is Jamie Lee Curtis?             Scorpio  
2  Which Lloyd Webber musical premiered in the US...    Sunset Boulevard  
3  Who was the next British Prime Minister after ...  Campbell-Bannerman  
4     Who had a 70s No 1 hit with Kiss You All Over?               Exile  
